# Diabetes Prediction

**The objective is to predict based on diagnostic measurements whether a patient has diabetes. <br>
Several constraints were placed on the selection of these instances from a larger database. In particular, all patients here are females at least 21 years old of Pima Indian heritage.**

#### Table of contents:
   - **Pregnancies**: Number of times pregnant
   - **Glucose**: Plasma glucose concentration a 2 hours in an oral glucose tolerance test
   - **BloodPressure**: Diastolic blood pressure (mm Hg)
   - **SkinThickness**: Triceps skin fold thickness (mm)
   - **Insulin**: 2-Hour serum insulin (mu U/ml)
   - **BMI**: Body mass index (weight in kg/(height in m)^2)
   - **DiabetesPedigreeFunction**: Diabetes pedigree function
   - **Age**: Age (years)
   - **Outcome**: Class variable (0 or 1)

Work plan:
1. [Study of general information.](#id1)
2. [Data preprocessing.](#id2)
3. [Exploratory data analysis.](#id3)
4. [Model building .](#id4)
5. [General conclusion.](#id5)

In [214]:
import pandas as pd
import numpy as np

from collections import Counter

import plotly.express as px
import plotly.figure_factory as ff
import plotly.graph_objects as go
from plotly.subplots import make_subplots
import seaborn as sns
import matplotlib.pyplot as plt
sns.set_theme()

import statsmodels.api as sm

from sklearn.linear_model import LogisticRegression
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.naive_bayes import GaussianNB
from sklearn.svm import SVC
from sklearn.svm import LinearSVC
from sklearn.neighbors import KNeighborsClassifier
from sklearn.linear_model import SGDClassifier

from sklearn import preprocessing
from sklearn.model_selection import train_test_split
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import cross_validate
from sklearn.metrics import classification_report
from sklearn.metrics import precision_recall_curve, auc, roc_auc_score
from sklearn.metrics import f1_score, precision_score, recall_score

import warnings
warnings.filterwarnings('ignore')

<a id="id1"></a>
## 1. Study of general information

In [215]:
data = pd.read_csv('diabetes.csv')
data.info()
data.head()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 768 entries, 0 to 767
Data columns (total 9 columns):
 #   Column                    Non-Null Count  Dtype  
---  ------                    --------------  -----  
 0   Pregnancies               768 non-null    int64  
 1   Glucose                   768 non-null    int64  
 2   BloodPressure             768 non-null    int64  
 3   SkinThickness             768 non-null    int64  
 4   Insulin                   768 non-null    int64  
 5   BMI                       768 non-null    float64
 6   DiabetesPedigreeFunction  768 non-null    float64
 7   Age                       768 non-null    int64  
 8   Outcome                   768 non-null    int64  
dtypes: float64(2), int64(7)
memory usage: 54.1 KB


,Pregnancies,Glucose,BloodPressure,SkinThickness,Insulin,BMI,DiabetesPedigreeFunction,Age,Outcome
0,6,148,72,35,0,33.6,0.627,50,1
1,1,85,66,29,0,26.6,0.351,31,0
2,8,183,64,0,0,23.3,0.672,32,1
3,1,89,66,23,94,28.1,0.167,21,0
4,0,137,40,35,168,43.1,2.288,33,1


In [216]:
data.describe()

,Pregnancies,Glucose,BloodPressure,SkinThickness,Insulin,BMI,DiabetesPedigreeFunction,Age,Outcome
count,768.000000,768.000000,768.000000,768.000000,768.000000,768.000000,768.000000,768.000000,768.000000
mean,3.845052,120.894531,69.105469,20.536458,79.799479,31.992578,0.471876,33.240885,0.348958
std,3.369578,31.972618,19.355807,15.952218,115.244002,7.884160,0.331329,11.760232,0.476951
min,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.078000,21.000000,0.000000
25%,1.000000,99.000000,62.000000,0.000000,0.000000,27.300000,0.243750,24.000000,0.000000
50%,3.000000,117.000000,72.000000,23.000000,30.500000,32.000000,0.372500,29.000000,0.000000
75%,6.000000,140.250000,80.000000,32.000000,127.250000,36.600000,0.626250,41.000000,1.000000
max,17.000000,199.000000,122.000000,99.000000,846.000000,67.100000,2.420000,81.000000,1.000000


In [217]:
data.duplicated().sum()

0

In [218]:
unique_number = data.nunique() \
                .reset_index() \
                .rename(columns={'index': 'Feature', 0: 'Count'})
fig = px.bar(unique_number, x='Count', y='Feature', 
             text='Count', template='plotly_dark',
             title='<b>Number of unique values<b>', height=800)
fig.update_traces(textposition='outside')
fig.update_layout(font_size=20,
                  font_family="San Serif")
fig.show()

In [219]:
count_target = data.Outcome \
    .value_counts() \
    .reset_index() \
    .rename(columns={'index': 'Diabetes', 'Outcome': 'Count'})
count_target['Diabetes'] = count_target['Diabetes'].map({0: 'No', 1: 'Yes'})

fig = px.pie(count_target, values='Count', names='Diabetes')
fig.update_layout(title='Distribution of the target variable',
                  font_family="San Serif",
                  font_size=24,
                  template='ggplot2', 
                  paper_bgcolor='lightgray')

fig.show()

<a id="id2"></a>
## 2. Data preprocessing

In [220]:
data.columns = data.columns.str.lower()
data.rename(columns={'outcome': 'diabetes', 
                     'diabetespedigreefunction': 'diabetes_pedigree_function'}, inplace=True)

Изучим распределения признаков с целью нахождения выбросов и аномальных значений.

In [221]:
def plot_hist(feature):
    fig = px.histogram(data, x=feature, marginal='box', nbins=len(data[feature].unique()))
    fig.update_xaxes(showgrid=False, zeroline=False)
    fig.update_layout(title=f'<b>Distribution of {feature}<b>', 
                      font_size=24,
                      font_family="San Serif",
                      font_color ='black',
                      template='plotly_dark',
                      paper_bgcolor="lightgray",
                      plot_bgcolor='lightgray')
    
    fig.show()

In [222]:
plot_hist('pregnancies')

In [223]:
plot_hist('glucose')

Значения "0" являются аномальными, т.к у человека не может быть уровня глюкозы равного 0. Заменим их на медианное значение глюкозы в зависимости от возраста и наличия диабета.

In [224]:
glucose = data.groupby(['diabetes', 'age'])['glucose'].median()

def fix_glucose(row):
    age = row['age']
    diabetes = row['diabetes']
    glucose_level = row['glucose']
    if glucose_level == 0:
        return glucose[diabetes, age]
    return glucose_level

data['glucose'] = data.apply(fix_glucose, axis=1)

In [225]:
plot_hist('bloodpressure')

Нулевые значения являются аномальными, т.к у человека не может быть диастолического равного 0. Заменим их на медианное значение давления в зависимости от возраста и наличия диабета.<br>
Для 72-летнего пациента нет данных о давлении для его возраста, поэтому в данном случае заменим нулевое значение на мелианное в зависимости от наличия диабета.

In [226]:
pressure = data.groupby(['diabetes', 'age'])['bloodpressure'].median()

def fix_pressure(row):
    age = row['age']
    diabetes = row['diabetes']
    blood_pressure = row['bloodpressure']
    if blood_pressure == 0:
        return pressure[diabetes, age]
    return blood_pressure

data['bloodpressure'] = data.apply(fix_pressure, axis=1)
data[data.index == 453].bloodpressure = data[data.diabetes == 0].bloodpressure.median()

In [227]:
plot_hist('bmi')

Нулевые значения признака bmi являются аномальными, т.к не бывает людей с нулевым весом. <br>
Заменим эти значения медианными в зависимости от наличия диабета.

In [228]:
bmi_by_diabetes = data.groupby('diabetes')['bmi'].median()

def fix_bmi(row):
    bmi = row['bmi']
    diabetes = row['diabetes']
    if bmi == 0:
        return bmi_by_diabetes[diabetes]
    return bmi

data['bmi'] = data.apply(fix_bmi, axis=1)

In [229]:
plot_hist('skinthickness')
skin_insulin_nulls = data[(data.skinthickness == 0) & (data.insulin == 0)].shape[0]
skin_nulls = data[data.skinthickness == 0].shape[0]
print(f'The number of zero values of the skinthickness attribute: {skin_nulls}')
print(f'The number of zero values of skinthickness and insulin attributes at the same time: {skin_insulin_nulls}')

The number of zero values of the skinthickness attribute: 227
The number of zero values of skinthickness and insulin attributes at the same time: 227


In [230]:
data[data.skinthickness != 0].corr()['skinthickness'].sort_values()

pregnancies                   0.100239
diabetes_pedigree_function    0.115016
insulin                       0.126423
age                           0.166816
bloodpressure                 0.227506
glucose                       0.231796
diabetes                      0.259491
bmi                           0.648495
skinthickness                 1.000000
Name: skinthickness, dtype: float64

Нулевые значения толщины кожной складки трицепса являются аномальными.<br>
Прослеживается взаимосвязь между индексом массы тела и толщиной кожной складки трицепса.<br>
Разобьём индексы массы тела на 5 равных по количеству категорий. Затем заменим нулевые значения на медианные по категории индекса массы тела и наличию диабета.

In [231]:
discretizer = KBinsDiscretizer(n_bins=5, encode='ordinal', strategy='quantile')
data['category_bmi'] = discretizer.fit_transform(data[['bmi']])

thickness = data[data.skinthickness != 0].groupby(['diabetes', 'category_bmi'])['skinthickness'].median()

def fix_skinthickness(row):
    category_bmi = row['category_bmi']
    diabetes = row['diabetes']
    skinthickness = row['skinthickness']
    if skinthickness == 0:
        return thickness[diabetes, category_bmi]
    return skinthickness

data['skinthickness'] = data.apply(fix_skinthickness, axis=1)
data = data[data.skinthickness != 99]

In [232]:
plot_hist('insulin')

In [233]:
data[data.insulin != 0].corr()['insulin'].sort_values()

pregnancies                   0.082171
bloodpressure                 0.098272
diabetes_pedigree_function    0.130395
skinthickness                 0.184888
age                           0.220261
bmi                           0.228519
category_bmi                  0.265316
diabetes                      0.303454
glucose                       0.581833
insulin                       1.000000
Name: insulin, dtype: float64

In [237]:
discretizer = KBinsDiscretizer(n_bins=5, encode='ordinal', strategy='quantile')
data['category_glucose'] = discretizer.fit_transform(data[['glucose']])

insulin = data[data.insulin != 0].groupby(['diabetes', 'category_glucose'])['insulin'].median()

def fix_insulin(row):
    category_glucose = row['category_glucose']
    diabetes = row['diabetes']
    insulin_level = row['insulin']
    if insulin_level == 0:
        return insulin[diabetes, category_glucose]
    return insulin_level

data['insulin'] = data.apply(fix_insulin, axis=1)

In [91]:
plot_hist('diabetes_pedigree_function')

In [92]:
plot_hist('age')

In [250]:
data.glucose = data.glucose.astype('int')
data.bloodpressure = data.bloodpressure.astype('int')
data.skinthickness = data.skinthickness.astype('int')
data.insulin = data.insulin.astype('int')
data.category_bmi = data.category_bmi.astype('int')
data.category_glucose = data.category_glucose.astype('int')

<a id="id3"></a>
## 3. Exploratory data analysis

In [252]:
corrs = data.corr()
ff.create_annotated_heatmap(
    z=corrs.values,
    x=list(corrs.columns),
    y=list(corrs.index),
    annotation_text=np.around(corrs.values, decimals=2),
    showscale=True)

### 3.1. Analysis of the relationship of each feature with the target

In [256]:
def plot_kde(feature):
    diabetes_yes = data[data.diabetes == 1][feature]
    diabetes_no = data[data.diabetes == 0][feature]

    fig = go.Figure()
    fig.add_trace(go.Violin(x=diabetes_yes, line_color='lightseagreen', name="Have diabetes", y0=0))
    fig.add_trace(go.Violin(x=diabetes_no, line_color='red', name= "No diabetes", y0=0))

    fig.update_traces(orientation='h', side='positive', meanline_visible=False)

    fig.update_layout(title=f'<b>The relationship between {feature} and the presence of diabetes<b>',
                      xaxis_title=feature,
                      xaxis_showgrid=False,
                      xaxis_zeroline=False,
                      titlefont={'size': 22},
                      width=900,
                      height=700,
                      template="plotly_dark",
                      showlegend=True,
                      paper_bgcolor="lightgray",
                      plot_bgcolor='lightgray', 
                      font=dict(
                          color ='black',
                          size=18)
                      )
    fig.show()

In [257]:
plot_kde('pregnancies')

**Чем больше количество беременностей в течение жизни, тем выше риск возникновения диабета. <br>
Это можно объяснить тем, что количество беременностей хорошо коррелирует с возрастом. В свою очередь, с увеличением возраста появляются провоцирующие диабет факторы.**

In [258]:
plot_kde('glucose')

In [309]:
fig = px.histogram(data, x='glucose', color='diabetes', marginal='box', barmode='group')

fig.update_layout(title='<b>Glucose distribution depending on the presence of diabetes<b>',
                  titlefont={'size': 24, 'family': 'San Serif'},
                  font_size=18,
                  height=650,
                  width=980,
                  template='plotly_dark')

fig.add_vline(x=199.91, line_width=2, line_dash='dot')
fig.add_vline(x=140.48, line_width=2, line_dash='dot')

fig.add_annotation(text='Diabetes<br> >200 mg/dl',x=209, y=45,showarrow=False,font_size=16)
fig.add_annotation(text='Prediabetes <br> between 140 and 199 mg/dl',x=170, y=45,showarrow=False,font_size=16)
fig.add_annotation(text='Normal <br> <140 mg/dl',x=65, y=45,showarrow=False,font_size=16)

fig.show()

Одним из важнейших показателей при подозрении на сахарный диабет является уровень глюкозы. С увеличением уровня глюкозы в крови возрастает вероятность наличия сахарного диабета. <br>
Стоит обратить внимание, что уровень глюкозы не является основанием для постановки диагноза. Как мы видим, в наборе данных есть пациенты с нормальным уровнем глюкозы, но болеющих сахарным диабетом. С возрастом норма глюкозы в крови увеличивается.

In [259]:
plot_kde('bloodpressure')

In [261]:
plot_kde('bmi')

In [318]:
fig = px.histogram(data, x='bmi', color='diabetes', marginal='box', barmode='group')

fig.update_layout(title='<b>Body mass index distribution depending on the presence of diabetes<b>',
                  titlefont={'size': 24, 'family': 'San Serif'},
                  font_size=18,
                  height=650,
                  width=980,
                  template='plotly_dark')

fig.add_vline(x=18.5, line_width=2, line_dash='dot')
fig.add_vline(x=24.9, line_width=2, line_dash='dot')
fig.add_vline(x=29.9, line_width=2, line_dash='dot')
fig.add_vline(x=39.9, line_width=2, line_dash='dot')

fig.add_annotation(text='Normal',x=22, y=37, showarrow=False, font_size=16)
fig.add_annotation(text='Over<br>weight', x=27.5, y=37, showarrow=False, font_size=16)
fig.add_annotation(text='Obesity', x=35, y=37, showarrow=False, font_size=16)
fig.add_annotation(text='Extreme obesity', x=58, y=37, showarrow=False, font_size=16)

fig.show()

**Вероятность наличия диабета увеличивается с ростом индекса массы тела.<br>
Люди с нормальным весом болеют диабетом реже остальных. Наибольший процент страдающих диабетом среди людей с ожирением и экстремальным ожирением.** 

In [260]:
plot_kde('skinthickness')

**Толщина кожной складки трицепса тесно коррелирует с индексом массы тела и нормируется по полу и возрасту. 
Распределение в зависимости от наличия диабета напоминает распределение индекса массы тела.<br>
С увеличением толщина кожной складки трицепса возрастает риск возникновения диабета.**

In [262]:
plot_kde('insulin')

**Инсулин – гормон, который вырабатывается в поджелудочной железе и отвечает за метаболизм глюкозы в организме.
Когда клетки нечувствительны к инсулину, глюкоза накапливается в крови, при этом ее избыток также превращается в жировые отложения. <br>
Анализ на инсулин в одиночном варианте малоинформативен. Как правило, назначается с анализом на глюкозу. При высоком уровне инсулина более вероятно наличие диабета.**

In [263]:
plot_kde('diabetes_pedigree_function')

**Генетическая предрасположенность к сахарному диабету оказывает влияние на вероятность появления данного заболевания. Чем выше значение признака, тем вероятнее возникновение заболевания.**

In [264]:
plot_kde('age')

**Чем старше человек, тем выше вероятность возникновения сахарного диабета.**

### 3.2. Analysis of other dependencies

In [253]:
data

,pregnancies,glucose,bloodpressure,skinthickness,insulin,bmi,diabetes_pedigree_function,age,diabetes,category_bmi,category_glucose
0,6,148,72,35,208,33.6,0.627,50,1,2,4
1,1,85,66,29,57,26.6,0.351,31,0,1,0
2,8,183,64,19,208,23.3,0.672,32,1,0,4
3,1,89,66,23,94,28.1,0.167,21,0,1,0
4,0,137,40,35,168,43.1,2.288,33,1,4,3
...,...,...,...,...,...,...,...,...,...,...,...
763,10,101,76,48,180,32.9,0.171,63,0,2,1
764,2,122,70,27,116,36.8,0.340,27,0,3,2
765,5,121,72,23,112,26.2,0.245,30,0,1,2
766,1,126,60,31,146,30.1,0.349,47,1,2,3
